In [1]:
import torchvision.models as models
import os
import sys
from torch.utils.data import DataLoader
from easydict import EasyDict as edict
import json
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch
import torch.nn.functional as F
import copy
from tqdm import tqdm
from torchvision import datasets, transforms

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import numpy as np

In [4]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [5]:
from data.dataset import ImageDataset

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
torch.cuda.device_count()

1

In [8]:
batch_size = 16

In [ ]:
model = models.resnet152(pretrained=True)

In [ ]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

In [ ]:
model.eval()

In [ ]:
model = model.to(device)

In [ ]:
pwd

In [30]:
transform = transforms.Compose(
        [
#             transforms.ToPILImage(),
#             transforms.Resize((256,256)),
            transforms.RandomResizedCrop((256),scale=(0.9,1)),
            transforms.RandomHorizontalFlip(),
#             transforms.RandomCrop(32, padding=4),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True),
        ])

In [34]:
dataset = datasets.ImageFolder('/home/diwenxu2/covid/cov_nor', transform=transform)

In [ ]:
dataset.class_to_idx

In [ ]:
pos = 0
neg = 0
for (d,s) in dataset.samples:
    if s==0:
        pos+=1
    if s==1:
        neg+=1
pos,neg

In [ ]:
dataset.samples = [(d, 1) if s == 0 else (d, 0) for d, s in dataset.samples]

In [ ]:
pos = 0
neg = 0
for (d,s) in dataset.samples:
    if s==1:
        pos+=1
    if s==0:
        neg+=1
pos,neg

In [ ]:
len(dataset)

In [ ]:
dataset.targets = torch.tensor(dataset.targets)
dataset.targets[dataset.targets==0] = 2
dataset.targets[dataset.targets==1] = 0
dataset.targets[dataset.targets==2] = 1

In [ ]:
pos = 0
neg = 0
for i in range(len(dataset.targets)):
    if dataset.targets[i] == 1:
        pos+=1
    if dataset.targets[i] == 0:
        neg+=1
pos,neg

In [ ]:
pos+neg

In [ ]:
tra_set, va_set = torch.utils.data.random_split(dataset, [10500, 3308])

In [ ]:
dataset_sizes = {'train':len(tra_set),'val':len(va_set)}

In [ ]:
dataset_sizes

In [ ]:
train_set = torch.utils.data.DataLoader(tra_set, batch_size=batch_size, shuffle=True)
val_set = torch.utils.data.DataLoader(va_set, batch_size=batch_size, shuffle=True)

In [ ]:
len(train_set),len(val_set)

In [ ]:
dataloaders = {'train':train_set,'val':val_set}

In [ ]:
inputs, classes = next(iter(dataloaders['val']))

In [ ]:
print(classes)

In [ ]:
imshow(inputs[2])

In [ ]:
imshow(inputs[10])

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=6)

In [ ]:
torch.save(model_ft.state_dict(), 'covid_classifier.ckpt')

In [4]:
model = models.resnet152(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.load_state_dict(torch.load('covid_classifier.ckpt'))

<All keys matched successfully>

In [5]:
model = model.cuda()

In [6]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [18]:
image_path = '/home/diwenxu2/xray_latent2im/model_Covid/stylegan_v2_xray_linear_lr0.0001_l2_w/images/w_1_seed12_Covid_max1.0_min0.0_sample3_0.64.png'

In [8]:
transform_ = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True),
        ])

In [9]:
from PIL import Image

# image = Image.open(image_path)

In [10]:
path = '/home/diwenxu2/xray_latent2im/20k_images/'
files = os.listdir(path)

In [13]:
def pred(model,img_path):
    image = Image.open(img_path).convert('RGB')
    transformed_img = transform_(image)
    transformed_img = transformed_img.to('cuda')
    image = transformed_img.unsqueeze(0).cuda()
    output = model(image)
    _,preds = torch.max(output,1)
    return preds

In [19]:
pred(model,'/home/diwenxu2/xray_latent2im/model_Covid/stylegan_v2_xray_linear_lr0.0001_l2_w/images/org_img/w_1_seed12_Covid_max1.0_min0.0_sample2_0.01_org.jpg')

tensor([0], device='cuda:0')

In [18]:
ratio = 0
for i in files[:1000]:
    img_path = path+i
    preds = pred(model,img_path)
    if preds == 1:
        ratio+=1
ratio/=1000
ratio

0.981

In [15]:
res = pred(model,path+files[0])

In [16]:
res==1

tensor([True], device='cuda:0')

In [31]:
transformed_img = transform_(image)
transformed_img = transformed_img.to('cuda')
image = transformed_img.unsqueeze(0).cuda()
output = model(image)

In [40]:
img_path = '/home/diwenxu2/xray_latent2im/model_Covid/stylegan_v2_xray_linear_lr0.0001_l2_w/images/w_1_seed12_Covid_max0.0_min0.0_sample4_0.23.png'

In [42]:
image = Image.open(img_path).convert('RGB')
transformed_img = transform_(image)
transformed_img = transformed_img.to('cuda')
image = transformed_img.unsqueeze(0).cuda()
output = model(image)
print(output)

tensor([[ 2.5315, -2.3207]], device='cuda:0', grad_fn=<AddmmBackward>)


In [37]:
res = model(img)
res

NameError: name 'img' is not defined

In [ ]:
_,preds = torch.max(res,1)
preds

In [ ]:
reg_json = '/home/diwenxu2/Chexpert/config/example.json'
with open(reg_json) as f:
    cfg = edict(json.load(f))

In [ ]:
dataloader_train = DataLoader(
        ImageDataset(cfg.train_csv, cfg, mode='train'),
        batch_size=batch_size, num_workers=12,
        drop_last=True, shuffle=True)

In [ ]:
len(dataloader_train)

In [ ]:
label_header = dataloader_train.dataset._label_header

In [ ]:
label_header

In [ ]:
dataloader_dev = DataLoader(
        ImageDataset(cfg.dev_csv, cfg, mode='dev'),
        batch_size=batch_size, num_workers=4,
        drop_last=False, shuffle=False)

In [ ]:
len(dataloader_dev)

In [ ]:
dev_header = dataloader_dev.dataset._label_header

In [ ]:
dev_header

In [ ]:
dataloaders = {'train':dataloader_train,'val':dataloader_dev}

In [ ]:
dataiter = iter(train_set)

In [ ]:
steps = len(dataloader_train)
steps

In [ ]:
images, labels = next(dataiter)

In [ ]:
imshow(images[2])

In [ ]:
images = images.to(device)
model = model.to(device)

In [ ]:
images.size()

In [ ]:
outputs = model(images)

In [ ]:
outputs.size()

In [ ]:
labels.size()

In [ ]:
labels[0]

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)

In [ ]:
label = torch.sigmoid(outputs[3].view(-1)).ge(0.5).float()

In [ ]:
label

In [ ]:
def get_loss(output, target, index, cfg, device):
    for num_class in cfg.num_classes:
        assert num_class == 1
    target = target[:, index].view(-1)
    pos_weight = torch.from_numpy(
        np.array(cfg.pos_weight,
                 dtype=np.float32)).to(device).type_as(target)
    if cfg.batch_weight:
        if target.sum() == 0:
            loss = torch.tensor(0., requires_grad=True).to(device)
        else:
            weight = (target.size()[0] - target.sum()) / target.sum()
            loss = F.binary_cross_entropy_with_logits(
                output[:,index].view(-1), target, pos_weight=weight)
    else:
        loss = F.binary_cross_entropy_with_logits(
            output[:,index].view(-1), target, pos_weight=pos_weight[index])

    return loss

In [ ]:
def train_model(model, optimizer, scheduler, device, num_epochs):
    dataset_sizes = {x: len(dataloaders[x]) for x in ['train', 'val']}
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in tqdm(range(num_epochs)):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                loss = 0
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    for t in range(5):
                        loss_t = get_loss(outputs, labels, t, cfg, device)
                        loss += loss_t

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                preds = torch.sigmoid(outputs).ge(0.5).float()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)/(len(preds)*len(preds[0]))
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model_ft = train_model(model, optimizer, exp_lr_scheduler,device,
                       num_epochs=15)

In [ ]:
torch.save(model_ft.state_dict(), 'vgg19.ckpt')

In [ ]:
vgg19 = models.vgg19(pretrained=False)
mod = list(vgg19.classifier.children())
mod.pop()
mod.append(torch.nn.Linear(4096, 2))
new_classifier = torch.nn.Sequential(*mod)
vgg19.classifier = new_classifier

In [ ]:
# vgg19 = nn.DataParallel(vgg19)
vgg19.load_state_dict(torch.load('covid_classifier.ckpt'))
vgg19 = vgg19.cuda().features.eval()

In [ ]:
model.train()
num_tasks = len(cfg.num_classes)
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader_train, 0)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].cuda(), data[1].cuda()
        outputs = model(inputs)
            
        loss = 0
        for t in range(num_tasks):
            loss_t = get_loss(outputs, labels, t, cfg)
            loss += loss_t
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')